In [30]:
import json
import shutil

import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import tiledb
import pdal
from pybabylonjs import Visualize3D

In [37]:
#!wget -nc "https://github.com/PDAL/data/blob/master/workshop/autzen.laz?raw=true" -O "autzen.laz"

In [4]:
#try:
#    shutil.rmtree("autzen")
#    pass
#except:
#    pass

In [36]:
pipeline1 = (
  pdal.Reader("autzen.laz") |
  pdal.Filter.stats() |
  pdal.Writer.tiledb(array_name="autzen1",chunk_size=10000000)
)

count = pipeline1.execute()  

In [37]:
pipeline2 = (
  pdal.Reader("autzen.laz") | 
  pdal.Writer.tiledb(array_name="autzen2",x_tile_size=10, y_tile_size=10, z_tile_size=1,chunk_size=10000000)
)

count = pipeline2.execute() 

In [11]:
pipeline3 = (
  pdal.Reader("autzen.laz") |
  pdal.Filter.stats() |
  pdal.Writer.tiledb(array_name="autzen3",chunk_size=10000000,data_tile_capacity=100000)
)

count = pipeline3.execute()  

In [9]:
pipeline4 = (
  pdal.Reader("autzen.laz") |
  pdal.Filter.stats() |
  pdal.Writer.tiledb(array_name="autzen4",chunk_size=100000,data_tile_capacity=100000)
)

count = pipeline4.execute()  

In [11]:
pipeline5 = (
  pdal.Reader("autzen.laz") |
  pdal.Filter.stats() |
  pdal.Writer.tiledb(array_name="autzen5",chunk_size=100000)
)

count = pipeline5.execute()  

## Point cloud

In [2]:
%%time
with tiledb.open("autzen1") as arr:
    df = pd.DataFrame(arr[636800:637800, 851000:853000, 406.14:615.26])

df.head()

CPU times: user 4.24 s, sys: 926 ms, total: 5.16 s
Wall time: 882 ms


,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,UserData,PointSourceId,GpsTime,Red,Green,Blue,X,Y,Z
0,1,1,1,1,0,2,-1.0,122,7331,248289.538944,111,112,98,637475.45,851804.30,415.49
1,1,1,1,1,0,2,10.0,122,7332,248680.116914,114,119,113,637174.99,852014.95,415.49
2,1,1,1,0,0,2,-12.0,124,7330,247564.723623,134,136,113,637371.69,851840.65,415.51
3,1,1,1,0,0,2,0.0,124,7331,248288.150659,138,145,124,637244.90,851873.25,415.51
4,1,1,1,1,0,2,-11.0,124,7330,247565.570015,112,114,108,637169.48,851998.29,415.51


In [3]:
data = {
    'X': df['X'],
    'Y': df['Y'],
    'Z': df['Z'],
    'Red': df['Red'] / 255.0,
    'Green': df['Green'] / 255.0,
    'Blue': df['Blue'] / 255.0
}
style = 'pointcloud'
width = 1200
height = 1000
z_scale = .2

In [31]:
Visualize3D.from_data(data=data,
                      style = 'pointcloud',
                        width = 800,
                        height = 600,
                        z_scale = .2)

BabylonPC(value={'style': 'pointcloud', 'width': 800, 'height': 600, 'z_scale': 0.2, 'extents': [636800.02, 63…

## MBRS

In [29]:
Visualize3D.from_array(array='autzen5',
                               style='mbrs',
                               width=800,
                               height=600,
                               z_scale = 0.2)

BabylonMBRS(value={'style': 'mbrs', 'width': 800, 'height': 600, 'z_scale': 0.2, 'extents': [635577.79, 639003…

In [40]:
Visualize3D.from_array(array='autzen3',
                               style='mbrs',
                               width=800,
                               height=600,
                               z_scale = 0.2)

BabylonMBRS(value={'style': 'mbrs', 'width': 800, 'height': 600, 'z_scale': 0.2, 'extents': [635577.79, 639003…

## Testing functions

In [34]:
def fragment_mbrs_df(array):
    
    fragments_info = tiledb.array_fragments(array,include_mbrs=True)
    
    df = pd.DataFrame()   

    f=0
    for fragment in fragments_info.mbrs:
        f+=1
        b=0
        for box in fragment:
            b+=1
            box_dict = {'fragment': f, 'box': b, 'xmin': box[0][0], 'xmax': box[0][1],
                    'ymin': box[1][0], 'ymax': box[1][1],
                    'zmin': box[2][0], 'zmax': box[2][1]}
            df = df.append(box_dict, ignore_index=True)

    df['H'] = (df['zmax']-df['zmin']) / (df['zmax'].max() - df['zmin'].min())
    df['W'] = (df['xmax']-df['xmin']) / (df['xmax'].max() - df['xmin'].min()) 
    df['D'] = (df['ymax']-df['ymin']) / (df['ymax'].max() - df['ymin'].min())
    df['X'] = (df['xmin']-df['xmin'].min() ) / (df['xmax'].max() - df['xmin'].min())
    df['Y'] = (df['ymin']-df['ymin'].min() ) / (df['ymax'].max() - df['ymin'].min())
    df['Z'] = (df['zmin']-df['zmin'].min() ) / (df['zmax'].max() - df['zmin'].min())    
    return df

In [35]:
df1 = fragment_mbrs_df("autzen1")
df1[df1['fragment']==2]

,fragment,box,xmin,xmax,ymin,ymax,zmin,zmax,H,W,D,X,Y,Z
100,2.0,1.0,637302.17,639003.01,852868.44,853480.45,408.43,418.76,0.049397,0.496459,0.131459,0.503330,0.856252,0.010951
101,2.0,2.0,637290.51,639003.14,852868.46,853488.11,418.76,421.69,0.014011,0.499901,0.133100,0.499927,0.856256,0.060348
102,2.0,3.0,637288.08,639003.50,852862.36,853472.83,421.69,423.79,0.010042,0.500715,0.131128,0.499218,0.854946,0.074359
103,2.0,4.0,637289.07,639002.20,852863.81,853466.76,423.79,425.00,0.005786,0.500047,0.129513,0.499507,0.855258,0.084401
104,2.0,5.0,637290.15,639003.73,852863.11,853468.26,425.00,440.98,0.076415,0.500178,0.129986,0.499822,0.855107,0.090187
105,2.0,6.0,637288.51,639003.31,852863.84,853464.33,440.98,460.89,0.095208,0.500534,0.128985,0.499343,0.855264,0.166603
106,2.0,7.0,637289.20,639003.63,852866.57,853463.24,460.89,540.26,0.379543,0.500426,0.128164,0.499545,0.855850,0.261811


In [36]:
df5 = fragment_mbrs_df("autzen5")
df5

,fragment,box,xmin,xmax,ymin,ymax,zmin,zmax,H,W,D,X,Y,Z
0,1.0,1.0,636075.30,637179.22,848935.20,849469.42,406.46,520.51,0.545381,0.322224,0.114750,0.145219,0.011395,0.001530
1,2.0,1.0,635577.79,636192.74,848965.87,850106.98,406.26,523.06,0.558531,0.179498,0.245110,0.000000,0.017983,0.000574
2,3.0,1.0,635577.88,635981.75,848979.71,850103.67,406.53,539.16,0.634229,0.117886,0.241426,0.000026,0.020956,0.001865
3,4.0,1.0,635756.95,636220.33,848970.83,850090.08,406.14,559.77,0.734650,0.135256,0.240414,0.052295,0.019048,0.000000
4,5.0,1.0,636030.47,636474.14,848963.34,850091.04,406.53,543.34,0.654218,0.129503,0.242229,0.132133,0.017440,0.001865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,103.0,1.0,637288.08,638028.11,852895.05,853488.06,408.43,488.68,0.383751,0.216008,0.127378,0.499218,0.861968,0.010951
103,104.0,1.0,637927.65,638678.40,852874.96,853465.58,409.57,538.85,0.618210,0.219137,0.126865,0.685902,0.857653,0.016402
104,105.0,1.0,638580.96,639003.73,852862.36,853446.82,410.82,538.68,0.611419,0.123403,0.125542,0.876597,0.854946,0.022379
105,106.0,1.0,637820.57,638683.00,853130.31,853472.46,409.55,534.48,0.597408,0.251735,0.073494,0.654647,0.912502,0.016306


In [41]:
df5['xmax'].max()

639003.73

In [ ]:
df5['xmin'].min()

639003.73

In [25]:
df5['xmax'].max()-df5['xmin'].min()

3425.939999999944